<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Working-Test---End-of-5-weeks-Project" data-toc-modified-id="Working-Test---End-of-5-weeks-Project-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Working Test - End of 5 weeks Project</a></span><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Loading-dataset" data-toc-modified-id="Loading-dataset-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Loading dataset</a></span></li><li><span><a href="#Cutting-into-small-pieces" data-toc-modified-id="Cutting-into-small-pieces-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Cutting into small pieces</a></span></li><li><span><a href="#Analysis-of-one-small-piece" data-toc-modified-id="Analysis-of-one-small-piece-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Analysis of one small piece</a></span><ul class="toc-item"><li><span><a href="#Display-of-the-chosen-chunk" data-toc-modified-id="Display-of-the-chosen-chunk-1.4.1"><span class="toc-item-num">1.4.1&nbsp;&nbsp;</span>Display of the chosen chunk</a></span></li><li><span><a href="#Optimization-of-deconvolution-parameters" data-toc-modified-id="Optimization-of-deconvolution-parameters-1.4.2"><span class="toc-item-num">1.4.2&nbsp;&nbsp;</span>Optimization of deconvolution parameters</a></span></li><li><span><a href="#Deconvolution-over-the-chunk" data-toc-modified-id="Deconvolution-over-the-chunk-1.4.3"><span class="toc-item-num">1.4.3&nbsp;&nbsp;</span>Deconvolution over the chunk</a></span></li><li><span><a href="#Storing-and-retrieving-the-results" data-toc-modified-id="Storing-and-retrieving-the-results-1.4.4"><span class="toc-item-num">1.4.4&nbsp;&nbsp;</span>Storing and retrieving the results</a></span></li><li><span><a href="#Displaying-the-results-of-deconvolution" data-toc-modified-id="Displaying-the-results-of-deconvolution-1.4.5"><span class="toc-item-num">1.4.5&nbsp;&nbsp;</span>Displaying the results of deconvolution</a></span></li><li><span><a href="#Peak-Picking" data-toc-modified-id="Peak-Picking-1.4.6"><span class="toc-item-num">1.4.6&nbsp;&nbsp;</span>Peak Picking</a></span></li></ul></li><li><span><a href="#Deconvolution-over-multiple-chunks-(and-the-whole-2D)" data-toc-modified-id="Deconvolution-over-multiple-chunks-(and-the-whole-2D)-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Deconvolution over multiple chunks (and the whole 2D)</a></span></li></ul></li></ul></div>

# Working Test - End of 5 weeks Project

## Imports

In [ ]:
from __future__ import print_function
from __future__ import division

In [ ]:
from bokeh.io import show,output_notebook

In [ ]:
output_notebook()

In [ ]:
try:
    from importlib import reload  # python 3
except:
    pass # python 2

In [ ]:
#import sys
#sys.path.append('/media/laura/5EA17E12330867F6/Work/1st Internship/')
import spike

In [ ]:
from spike import FTICR

In [ ]:
%matplotlib notebook

In [ ]:
import sys
sys.path.append('/home/laura/Desktop/isotopic-deconv-basic-code')
import os
try:
    from importlib import reload
except:
    pass
import numpy as np
import scipy
from scipy.signal import fftconvolve
import matplotlib.mlab as mlab
from isotope import isotopes as iso
iso.THRESHOLD = 1E-5 # devrait être 1E-5 ! mais plus rapide
from isotope.proteins import averagine
from isotope import Fstore
import Deconvolution_fast as dec
reload(dec)
import isotopic_analysis as ia
reload(ia)
import tools_2DFTICR_Data as tools2D
reload(tools2D)
import PDS
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from scipy.optimize import curve_fit
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
reload(ia)
dec.standard = False

In [ ]:
reload(dec)

In [ ]:
dec.memoize_clear()

## Loading dataset

In [ ]:
data_name = "/home/laura/Desktop/isotopic-deconv-basic-code_local/Data/yeast_2D_000002_dn200_mr.msh5"

To see the entire spectrum in a low resolution

Importation of data - Complete yeast extract 2D FT-ICR MS Spectrum

In [ ]:
d = FTICR.FTICRData(name=data_name, mode="onfile") #Mode onfile avoid taking too much space on the computer memory

In [ ]:
d._absmax = 76601098.15623298

## Cutting into small pieces

Creation of all parts from this spectrum - Each part is 1024x256 points sized. The coordinates are contained in Tabl.

In [ ]:
Tabl = tools2D.pieceofdata(d,3*1024,256)

In [ ]:
print(len(Tabl)) #To know how many parts need to be deconvoluted

In [ ]:
for N in range(len(Tabl)):
    zoom = list(tools2D.isz(d,Tabl[N]))
    Y = [zoom[1],zoom[0],zoom[1],zoom[0]]
    X = [zoom[3],zoom[3],zoom[2],zoom[2]]
    plt.scatter(X,Y, s=0.5, c="red", alpha=0.5)

The analysis will then be performed on only one part from the previous Tabl. Only to demonstrate that it works for one part and that the next step is only to loop over all parts.

## Analysis of one small piece

Chosen part : Tabl[N]

### Display of the chosen chunk

In [ ]:
N = 5
zoom = Tabl[N]

In [ ]:
d.unit = 'm/z' #To make sure we work in points
d.display(scale=25, zoom=tools2D.isz(d,zoom))

In [ ]:
zoom_mz = tools2D.isz(d,zoom)

In [ ]:
zoom_mz_safe = [zoom_mz[0]+2,zoom_mz[1]-2,zoom_mz[2]+2,zoom_mz[3]-2]

In [ ]:
zoom_safe = list(tools2D.mztoi(d,zoom_mz_safe))

In [ ]:
print(zoom,list(zoom_mz),zoom_mz_safe,zoom_safe)

In [ ]:
d.unit = 'points' #To make sure we work in points
d.display(scale=5, zoom=zoom)

In [ ]:
d.unit='m/z'
d.bokeh(zoom=tools2D.isz(d,zoom),scale = 0.4, show=True)

### Optimization of deconvolution parameters

In [ ]:
#Boucle pour voir évolution khi2 et maxres en fonction des valeurs de width1 width2
khi2 = []
maxres = []
coord = []
import numpy as np
Width1 = [i for i in np.arange(0.8,3.0,0.4)]
Width2 = [i for i in np.arange(0.0,0.2,0.04)]
for i in Width1:
    for j in Width2:
        result = tools2D.analysis(d,zoom, width2=j, width1=i)
        
        khi2 += [np.sqrt(result.Khi2)]
        maxres += [result.result.max()]
        w1,w2,k = i, j, np.sqrt(result.Khi2)
        coord.append([w1,w2,k])
        print (coord)

In [ ]:
#Affichage sqrt(khi2) en fonction de width1 width2
X = []
Y = []
Z = []
for n in coord:
    X.append(n[0])
    Y.append(n[1])
    Z.append(n[2])
plt.gca(projection='3d').plot(X,Y,Z,'--oc')
plt.xlabel('Width1')
plt.ylabel('Width2')

In [ ]:
#Affichage contourf khi2 
Z = np.array(Z)
Z = Z.reshape((len(Width1), len(Width2)))
X, Y = np.meshgrid(Width2,Width1)
CS = plt.contourf(X, Y, Z)
plt.xlabel('Width2')
plt.ylabel('Width1')
cbar = plt.colorbar(CS)
cbar.ax.set_ylabel('sqrt(khi2)')

In [ ]:
#Affichage contourf maxres
b = np.array(maxres)
b = b.reshape((len(Width1), len(Width2)))
X, Y = np.meshgrid(Width2,Width1)
CS2=plt.contourf(X, Y, b)
plt.xlabel('Width2')
plt.ylabel('Width1')
cbar = plt.colorbar(CS2)
cbar.ax.set_ylabel('MaxRes')

In [ ]:
#Affichage maxres en fonction de width1 width2
X = []
Y = []
for n in coord:
    X.append(n[0])
    Y.append(n[1])
plt.gca(projection='3d').plot(X,Y,maxres,'--ob')
plt.xlabel('Width1')
plt.ylabel('Width2')

In [ ]:
#Affichage de sqrt(khi2) en fonction de maxres
Z = []
for n in coord:
    Z.append(n[2])
plt.plot(Z,maxres,'*')
plt.xlabel('Sqrt(khi2)')
plt.ylabel('MaxRes')

In [ ]:
#Boucle pour voir évolution khi2 et maxres en fonction des valeurs de noisefactor
khi2 = []
maxres = []
coord = []
import numpy as np
for i in np.arange(0.1,10.5,0.4):
        result = tools2D.analysis(d,[3584, 3840, 51691, 52715], noisefactor = i, width2=0.1, width1=1.4)
        khi2 += [np.sqrt(result.Khi2)]
        maxres += [result.result.max()]
        noisefact,k = i, np.sqrt(result.Khi2)
        coord.append([noisefact,k])
        print (coord)

In [ ]:
noisefact = []
k = []
for n in coord:
    noisefact.append(n[0])
    k.append(n[1])
plt.plot(noisefact,k,'--c.',)
plt.xlabel('NoiseFactor')
plt.ylabel('Sqrt(khi2)')

In [ ]:
plt.plot(noisefact,maxres,'--g.',)
plt.xlabel('NoiseFactor')
plt.ylabel('maxres')
axes = plt.gca()
axes.yaxis.set_label_position('right')
axes.yaxis.set_ticks_position('right')

### Deconvolution over the chunk

In [ ]:
reload(tools2D)
reload(dec)

In [ ]:
%load_ext memory_profiler

In [ ]:
d.unit='points'
%time S = tools2D.analysis(d,zoom,width1=2.0,width2=0.04, noisefactor=0.9)

In [ ]:
print(dec.Samplez2D.__dict__)
dec.Samplez2D.cache.clear()

In [ ]:
print(dec.Samplez2Dt.__dict__)
dec.Samplez2Dt.cache.clear()

In [ ]:
print(dec.FF.Memo)

In [ ]:
dec.FF.Memo = {}

In [ ]:
print(open('mprof0', 'r').read())

In [ ]:
d.buffer.shape

In [ ]:
S.zone

In [ ]:
S.result.shape

### Storing and retrieving the results

In [ ]:
tools2D.store("Results_{0}_{1}_{2}_{3}.h5".format(*Tabl[N]),S.result)

In [ ]:
Solved = tools2D.read("Results_{0}_{1}_{2}_{3}.h5".format(*Tabl[N]))
Solved.shape

### Displaying the results of deconvolution

S is then a rich object containing a lot of information, including the results of deconvolution.

In [ ]:
#S.spectrum contain the original part of spectrum
X, Y = np.meshgrid(S.axeM1, S.axeM2)
ax2 = plt.subplot(1,1,1, projection='3d')
ax2.plot_surface(X,Y,S.spectrum.T, color='b', cmap=cm.coolwarm,  linewidth=0.05, cstride=2, rstride=2);

In [ ]:
#S.result contain the results of deconvolution. It is a 4D array as it contains information for all Z values requested.
#Here Z = 1,2 for both axis.
X, Y = np.meshgrid(S.axeM1, S.axeM2) 
ax = plt.subplot(1,1,1, projection='3d')   
ax.plot_surface(X,Y,(S.result[1,0,:,:]).T, color='b', cmap=cm.coolwarm, linewidth=0.05, cstride=2, rstride=2, vmax=1E9);

In [ ]:
X, Y = np.meshgrid(S.axeM1, S.axeM2) 
ax = plt.subplot(1,1,1, projection='3d')   
ax.plot_surface(X,Y,(S.result[0,0,:,:]).T, color='b', cmap=cm.coolwarm, linewidth=0.05, cstride=2, rstride=2, vmax=1E9);

In [ ]:
X, Y = np.meshgrid(S.axeM1, S.axeM2) 
ax = plt.subplot(1,1,1, projection='3d')   
ax.plot_surface(X,Y,(S.result[0,1,:,:]).T, color='b', cmap=cm.coolwarm, linewidth=0.05, cstride=2, rstride=2, vmax=1E9);

In [ ]:
X, Y = np.meshgrid(S.axeM1, S.axeM2) 
ax = plt.subplot(1,1,1, projection='3d')   
ax.plot_surface(X,Y,(S.result[1,1,:,:]).T, color='b', cmap=cm.coolwarm, linewidth=0.05, cstride=2, rstride=2, vmax=1E9);

Here width values haven't been optimized, it should be as we see that the deconvolution is not perfect. See just bellow the convergence value sqrt(Khi2) that has been computed during the deconvolution process. It can be closest to 1 than that.

In [ ]:
np.sqrt(S.Khi2) #represent the convergence (the closest to 1, the better)

### Peak Picking

The next step is to perform a peak picking on the deconvoluted data. WARN: As the pp() function is not finished, the list of peaks is performed only for one couple of Z values results - See the code of the function.

In [ ]:
d.display(scale=5, zoom=zoom)

In [ ]:
reload(spike)

In [ ]:
threshold = 2E6
threshold

In [ ]:
d.pp(threshold=threshold, zoom=tools2D.mztoi(d,(400,700,300,1200)))

In [ ]:
len(d.peaks)

In [ ]:
reload(tools2D)

In [ ]:
import numpy

In [ ]:
#PeakPicking on deconvoluted
threshold = 1E7
listpkF1_00, listpkF2_00, listint_00 = tools2D.pp(S,0,0,threshold = threshold)
print("00: ",listpkF1_00, listpkF2_00, listint_00,"\n")
listpkF1_11, listpkF2_11, listint_11 = tools2D.pp(S,1,1,threshold = threshold)
print("11: ",listpkF1_11, listpkF2_11, listint_11,"\n")
listpkF1_10, listpkF2_10, listint_10 = tools2D.pp(S,1,0,threshold = threshold)
print("10: ",listpkF1_10, listpkF2_10, listint_10,"\n")
listpkF1_01, listpkF2_01, listint_01 = tools2D.pp(S,0,1,threshold = threshold)
print("01: ",listpkF1_01, listpkF2_01, listint_01,"\n")

In [ ]:
d.display(scale=1, zoom=zoom)
d.display_peaks(zoom=zoom)

In [ ]:
d.display(scale=1, zoom=zoom)
plt.plot(listpkF2_10, listpkF1_10,'xr')
plt.plot(listpkF2_00, listpkF1_00,'+b')
plt.plot(listpkF2_01, listpkF1_01,'xr')
plt.plot(listpkF2_11, listpkF1_11,'+b')
plt.show()

In [ ]:
#3D Representation of the points found ON DECONVOLUTION RESULT.
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot(listpkF1_00,listpkF2_00,listint_00,'xg')
ax.plot(listpkF1_11,listpkF2_11,listint_11,'xy')
ax.plot(listpkF1_10,listpkF2_10,listint_10,'xk')
ax.plot(listpkF1_01,listpkF2_01,listint_01,'xr')
plt.show()

In [ ]:
#PP Before deconvolution
threshold=2E6
x,y,z = d.peaks2d(threshold,zoom)
plt.gca(projection='3d').plot(x,y,z,'+b')

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')
X, Y = np.meshgrid(S.axeM1, S.axeM2) 
ax.plot_surface(X,Y,(S.result[1,0,:,:]).T, color='b', cmap=cm.coolwarm, linewidth=0.05, cstride=2, rstride=2, vmax=1E9)
ax.plot([d.axis1.itomz(p) for p in listpkF1_10],[d.axis2.itomz(p) for p in listpkF2_10],listint_10,'.r')
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')
X, Y = np.meshgrid(S.axeM1, S.axeM2) 
ax.plot_surface(X,Y,(S.result[0,0,:,:]).T, color='b', cmap=cm.coolwarm, linewidth=0.05, cstride=2, rstride=2, vmax=1E9)
ax.plot([d.axis1.itomz(p) for p in listpkF1_00],[d.axis2.itomz(p) for p in listpkF2_00],listint_00,'.c')
plt.show()

In [ ]:
d.unit='m/z'
d.display(zoom=tools2D.isz(d,zoom),scale = 5)
plt.plot([d.axis2.itomz(p) for p in listpkF2_10],[d.axis1.itomz(p) for p in listpkF1_10], 'xr')
plt.plot([d.axis2.itomz(p) for p in listpkF2_00],[d.axis1.itomz(p) for p in listpkF1_00], '+b')
plt.plot([d.axis2.itomz(p) for p in listpkF2_11],[d.axis1.itomz(p) for p in listpkF1_11], '*k')
plt.plot([d.axis2.itomz(p) for p in listpkF2_01],[d.axis1.itomz(p) for p in listpkF1_01], '.c')
plt.xlabel("m/z")
plt.ylabel("m/z")


NB.: Resulting coordinates are in points, not in m/z.

In [ ]:
listpkF1bis_10,listpkF2bis_10,listintbis_10,widthF1_10,widthF2_10 = tools2D.centroid2D_OnDeconvoluted(S,1,0,listpkF1_10, listpkF2_10, listint_10,npoints_F1=3, npoints_F2=3)

In [ ]:
listpkF1bis_00,listpkF2bis_00,listintbis_00,widthF1_00,widthF2_00 = tools2D.centroid2D_OnDeconvoluted(S,0,0,listpkF1_00, listpkF2_00, listint_00,npoints_F1=3, npoints_F2=3)

In [ ]:
len(listpkF2bis_10)

In [ ]:
d.unit='m/z'
plt.figure()
plt.plot([d.axis2.itomz(p) for p in listpkF2_10],[d.axis1.itomz(p) for p in listpkF1_10], 'xr') #Before centroid
plt.plot([d.axis2.itomz(p) for p in y],[d.axis1.itomz(p) for p in x], 'xr') #PeakPicking before deconvolution
plt.xlabel("m/z")
plt.ylabel("m/z")
plt.xlim(885,908)
plt.ylim(535,595)
plt.title("Peak Picking before deconvolution")
plt.figure()
plt.plot([d.axis2.itomz(p) for p in listpkF2bis_10],[d.axis1.itomz(p) for p in listpkF1bis_10], '.g') #After centroid
plt.xlabel("m/z")
plt.ylabel("m/z")
plt.xlim(885,908)
plt.ylim(535,595)
plt.title("Peak Picking on deconvolution results")
d.display(zoom=tools2D.isz(d,zoom),scale = .5) #Original Data


In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')
X, Y = np.meshgrid(S.axeM1, S.axeM2) 
ax.plot_surface(X,Y,(S.result[1,0,:,:]).T, color='b', cmap=cm.coolwarm, linewidth=0.05, cstride=2, rstride=2, vmax=1E9)
ax.plot([d.axis1.itomz(p) for p in listpkF1bis_10],[d.axis2.itomz(p) for p in listpkF2bis_10],listint_10,'.r')
ax.plot([d.axis1.itomz(p) for p in listpkF1_10],[d.axis2.itomz(p) for p in listpkF2_10],listint_10,'xg')
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')
X, Y = np.meshgrid(S.axeM1, S.axeM2) 
ax.plot_surface(X,Y,(S.result[0,0,:,:]).T, color='b', cmap=cm.coolwarm, linewidth=0.05, cstride=2, rstride=2, vmax=1E9)
ax.plot([d.axis1.itomz(p) for p in listpkF1bis_00],[d.axis2.itomz(p) for p in listpkF2bis_00],listint_00,'.c')
plt.show()

Below are the masses of fragments and parents respectively

## Deconvolution over multiple chunks (and the whole 2D)

Use of the multi_analysis function - TO DO: ADJUST WIDTHS VALUES ACCORDING TO THE ZONE.

In [ ]:
reload(dec)
reload(tools2D)

In [ ]:
%mprun?

In [ ]:
%mprun -T mprof0 -r -c -f tools2D.multi_analysis(d,Tabl[5:6],iterations=100,width2=0.1)

In [ ]:
print(open('mprof0', 'r').read())

In [ ]:
del Tabl

In [ ]:
reload(dec)
reload(tools2D)

In [ ]:
%time tools2D.multi_analysis(d, Tabl[5:6], iterations=100,width2=0.1)

In [ ]:
pk_21_f2 = []
pk_21_f1 = []

pk_11_f2 = []
pk_11_f1 = []

pk_22_f2 = []
pk_22_f1 = []

pk_12_f2 = []
pk_12_f1 = []

for key in d.pks:
    if "Z12_Z21" in key:
        for e in d.pks[key][3]: 
            pk_21_f1.append(e)
        for e in d.pks[key][4]:
            pk_21_f2.append(e)
    if "Z11_Z21" in key:
        for e in d.pks[key][3]: 
            pk_11_f1.append(e)
        for e in d.pks[key][4]:
            pk_11_f2.append(e)
    if "Z12_Z22" in key:
        for e in d.pks[key][3]: 
            pk_22_f1.append(e)
        for e in d.pks[key][4]:
            pk_22_f2.append(e)
    if "Z11_Z22" in key:
        for e in d.pks[key][3]: 
            pk_12_f1.append(e)
        for e in d.pks[key][4]:
            pk_12_f2.append(e)


In [ ]:
d.pks

In [ ]:
d.unit='m/z'
fig = plt.figure(figsize=(18, 12))
ax = fig.add_subplot(111)
plt.plot(pk_21_f2,pk_21_f1,'xg')
plt.plot(pk_11_f2,pk_11_f1,'.r')
plt.plot(pk_12_f2,pk_12_f1,'.r')
plt.plot(pk_22_f2,pk_22_f1,'.r')
d.display(scale=1, zoom=tools2D.isz(d,(1652,2164,36000,38048)),figure=ax)

In [ ]:
DI = MR_interact(data_name,
                report=False,   # inhibits parameter printing
                show=False,     # does not display on start-up
                figsize=(20,15),# Size of initial display (in cm)
                Debug=False     # Enables live debugging if True
               )
DI._zoom = (400,700,137,1500)     # set the initial zoom view, in m/z (F1low , F1High , F2low , F2High)
DI.scale = 5.0                      # set the initial scale
DI.show()                           # and finally show

In [ ]:
print(tools2D.isz(d,Tabl[533]))

In [ ]:
plt.plot(pk_21_f2,pk_21_f1,'xg')
plt.plot(pk_11_f2,pk_11_f1,'.r')
plt.plot(pk_12_f2,pk_12_f1,'+b')
plt.plot(pk_22_f2,pk_22_f1,'*k')

In [ ]:
d.unit='points'

In [ ]:
reload(dec)

In [ ]:
%time tools2D.multi_analysis(d, Tabl[602:605], iterations=100,width2=1.0)

In [ ]:
d.pks

In [ ]:
d.unit='m/z'

In [ ]:
N602_11 = d.pks["Peaks_Zone230425154871949743_Z11_Z21"]
N602_12 = d.pks["Peaks_Zone230425154871949743_Z11_Z22"]
N602_21 = d.pks["Peaks_Zone230425154871949743_Z12_Z21"]
N602_22 = d.pks["Peaks_Zone230425154871949743_Z12_Z22"]
fig, axs = plt.subplots(1, 4, figsize=(16, 4))
axs[0] = plt.plot(N602_11[4],N602_11[3],'.r')
#axs[0].set_title("Z: 1 1")
axs[1] = plt.plot(N602_12[4],N602_12[3],'.r')
#axs[1].set_title("Z: 1 2")
axs[2] = plt.plot(N602_21[4],N602_21[3],'.r')
#axs[2].set_title("Z: 2 1")
axs[3] = plt.plot(N602_22[4],N602_22[3],'.r')
#axs[3].set_title("Z: 2 2")

In [ ]:
N603_11 = d.pks["Peaks_Zone230425154974350767_Z11_Z21"]
N603_12 = d.pks["Peaks_Zone230425154974350767_Z11_Z22"]
N603_21 = d.pks["Peaks_Zone230425154974350767_Z12_Z21"]
N603_22 = d.pks["Peaks_Zone230425154974350767_Z12_Z22"]
fig, axs = plt.subplots(1, 4, figsize=(16, 4))
axs[0] = plt.plot(N603_11[4],N603_11[3],'.r')
#axs[0].set_title("Z: 1 1")
axs[1] = plt.plot(N603_12[4],N603_12[3],'.r')
#axs[1].set_title("Z: 1 2")
axs[2] = plt.plot(N603_21[4],N603_21[3],'.r')
#axs[2].set_title("Z: 2 1")
axs[3] = plt.plot(N603_22[4],N603_22[3],'.r')
#axs[3].set_title("Z: 2 2")

In [ ]:
print(N603_22[4],N603_22[3])

In [ ]:
d.unit='m/z'
d.display(zoom=[530,540,765,775],scale = 1.)

In [ ]:
N602 = d.pks["Peaks_Zone230425154871949743_Z12_Z22"]
N603 = d.pks["Peaks_Zone230425154974350767_Z12_Z22"]
N604 = d.pks["Peaks_Zone230425155076751791_Z12_Z22"]

In [ ]:
d.unit='m/z'
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(N602[4],N602[3],'.r')
plt.plot(N603[4],N603[3],'.r')
plt.plot(N604[4],N604[3],'.r')
#d.display(zoom=[538,585,855,910],scale = 1., figure = ax)

In [ ]:
a = b = np.ones(100)

In [ ]:
a = {'a':123, 'b':'abc'}
a

In [ ]:
a.clear()
a